In [1]:
import pandas as pd
from django_pandas.io import read_frame

In [20]:
from mec.models import Contribution, Address, Contributor, District

In [22]:
Address.objects.filter(coordinates__contained=District.objects.get(number=95).boundaries)

<QuerySet [<Address: 4656 Baumgartner Rd, Saint Louis, MO 63129>, <Address: 3114 Woodbridge Est Drive, Oakville, MO 63129>, <Address: 2957 Sean Pkwy, St Louis, MO 63129-4512>, <Address: 5909 Cardinal Creek Dr, St Louis, MO 63129-4662>, <Address: 2404 Boardwalk Place Drive, St Louis, MO 63129-4853>, <Address: 5806 Southcrest Way, St Louis, MO 63129>, <Address: 3775 Gumtree Ln, St Louis, MO 63129>, <Address: 7186 Christopher, St Louis, MO 63129>, <Address: 3520 Jeffco Blvd, Arnold, MO 63010>, <Address: 4029 Jeffco Blvd, Arnold, MO 63010>, <Address: 5921 Briarmist Pl, Saint Louis, MO 63128-4427>, <Address: 810 Jeffco Blvd, Arnold, MO 63010>, <Address: Mehlville, St Louis, MO 63129>, <Address: 6134 Pathfinder Dr, St Louis, MO 63129>, <Address: 5091 Baumgartner Road, St Louis, MO 63129>, <Address: 4173 Fairburn Ct, Saint Louis, MO 631297238>, <Address: 4949 Karington Place Dr, Saint Louis, MO 631297101>, <Address: 5132 Crosswood Dr, St Louis, MO 63129>, <Address: 2687 Sanders, St Louis, MO 

In [26]:
donors_95th = Contributor.objects.filter(address__coordinates__contained=District.objects.get(number=95).boundaries)

In [34]:
contributions_95th = Contribution.objects.filter(contributor__address__coordinates__contained=District.objects.get(number=95).boundaries)

In [43]:
df = read_frame(contributions_95th, fieldnames=[
    'date',
    'amount',
    'recipient__name',
    'contributor__committee__name',
    'contributor__company__name',
    'contributor__individual__first_name',
    'contributor__individual__last_name',
    'contributor__individual__employer',
    'contributor__individual__occupation',
    'contributor__address__address1'
    
])

In [44]:
df = df.rename(columns={'amount':'Amount',
                                 'date':'Date',
                                 'recipient__name':'Recipient',
                                 'contributor__individual__first_name':'First',
                                'contributor__individual__last_name':'Last',
                                'contributor__individual__employer':'Employer',
                                'contributor__individual__occupation':'Occupation',
                                'contributor__address__address1':'Address'})

In [50]:
df1 = df.astype({'Amount':'float'})
df1 = df1.groupby(['First','Last','Employer','Occupation','Address']).agg({'Amount':['mean','count','sum'],'Recipient': lambda x: pd.Series.mode(x)[0], 'Date':'max'}).sort_values(by=('Amount','count'),ascending=False)
df1.columns = df1.columns.get_level_values(1)
df1 = df1.rename(columns={'sum':'Total Amount ($)','count':'# of Donations','mean':'Avg Donation ($)','<lambda>':'Most Common Recipient','max':'Most Recent Donation'})
df1.sort_values(by='# of Donations')
df1.to_csv('95th- initial list.csv')

In [49]:
df1.to_csv('mo_83_clean.csv')

In [270]:
df2 = pd.read_csv('vanphonenumbers.csv').rename(columns={'Last Name':'Last','First Name':'First'})
df4 = pd.read_csv('vanphonenumbers2.csv').rename(columns={'Last Name':'Last','First Name':'First'})
df5 = pd.read_csv('vanphonenumbers3.csv',low_memory=False).rename(columns={'Last Name':'Last','First Name':'First'})
df2['Phone Number'] = df2['Phone Number'].astype(str)
df4['Phone Number'] = df4['Phone Number'].astype(str)
# df5['Phone Number'] = df5['Phone Number'].astype(str)
# df6 = df2.merge(df4,left_on='Phone Number')
df6 = pd.concat([df2,df4,df5]).drop_duplicates()

In [271]:
df2 = df6
df2 = df2[['First','Last','Phone Number','Is Cell?','Is Preferred?']]
df2['First'] = df2['First'].apply(lambda x: x.split()[0])
# df4 = df4[['First','Last','Phone Number','Is Cell?','Is Preferred?']]
# df4['First'] = df4['First'].apply(lambda x: x.split()[0])
# df5 = df5[['First','Last','Phone Number','Is Cell?','Is Preferred?']]
# df5['First'] = df5['First'].apply(lambda x: x.split()[0])
df3 = df1.reset_index().merge(df2,on=['First','Last'],how='left') #.merge(df4,on=['First','Last',],how='left').merge(df5,on=['First','Last',],how='left')
df3['Avg Donation ($)'] = df3['Avg Donation ($)'].round(2)
df3['Total Amount ($)'] = df3['Total Amount ($)'].round().astype(int)
# df3['Phone Number'] = df3['Phone Number_x'].combine_first(df3['Phone Number_y'])
# df3 = df3.drop(columns=['Phone Number_x', 'Phone Number_y'])
# df3['Is Cell?'] = df3['Is Cell?_x'].combine_first(df3['Is Cell?_y'])
# df3 = df3.drop(columns=['Is Cell?_x', 'Is Cell?_y'])
# df3['Is Preferred?'] = df3['Is Preferred?_x'].combine_first(df3['Is Preferred?_y'])
# df3 = df3.drop(columns=['Is Preferred?_x', 'Is Preferred?_y'])
df3 = df3.fillna('NA')
df3

/home/tyler/.virtualenvs/thesis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,First,Last,Employer,Occupation,Address,Avg Donation ($),# of Donations,Total Amount ($),Most Common Recipient,Most Recent Donation,Phone Number,Is Cell?,Is Preferred?
0,Jeanne,Kirkton,Requested,Requested,659 Tuxedo,130.91,92,12044,IstMo Nominee Pac Number Two,2020-03-31,NA,NA,NA
1,Michele,Bunch,Farmers Insurance,Field Claims Supervisor,3721 Regal Pl,7.68,85,652,Feapac Of Missouri,2019-03-14,13144060059,Verified Cell,0
2,Michele,Bunch,Farmers Insurance,Field Claims Supervisor,3721 Regal Pl,7.68,85,652,Feapac Of Missouri,2019-03-14,16363299572,Likely Not a Cell,0
3,Beth,Newman,Kipp Saint Louis,Speech Language Pathologist,7401 Gayola Place,32.44,85,2757,Elad Gross For Missouri,2020-03-31,13146013113,Likely Not a Cell,0
4,Beth,Newman,Kipp Saint Louis,Speech Language Pathologist,7401 Gayola Place,32.44,85,2757,Elad Gross For Missouri,2020-03-31,13143153425,Likely Cell,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,Jim,Schallom,Sherwood Forest,Nonprofit Management,6029 Hartford Street,100.00,1,100,Team Clancy,2018-06-08,NA,NA,NA
3919,Jim,Mittino,City Of St Louis,Manager,5032 Murdoch Ave,100.00,1,100,Citizens For Tishaura O Jones,2014-03-14,NA,NA,NA
3920,Jim,Mcgivney,Retired,Retired,346 Tulip,75.00,1,75,Sarah Unsicker For State Representative,2016-10-03,NA,NA,NA
3921,Jim,Mccarthy,Not Employed,Not Employed,4964 Fyler Av Apt A,50.00,1,50,Megan Green For St Louis,2019-02-20,NA,NA,NA


In [276]:
len(df3[df3['Phone Number']!='NA']) #.drop_duplicates(subset=['First','Last']))

2891

In [273]:
df3.to_csv('MO83_w_numbers.csv',index=False)

In [278]:
pd.unique(df3[['Last','First']])

ValueError: could not broadcast input array from shape (3923,2) into shape (3923)

In [289]:
df3.drop_duplicates(subset=['First','Last','Employer','Occupation']).reset_index().loc[500,['Last','First']]

Last     Larose
First      Lisa
Name: 500, dtype: object